In [1]:
import socketio
import eventlet
import numpy as np
from flask import Flask
from keras.models import load_model
import base64
from io import BytesIO
from PIL import Image
import cv2







Using TensorFlow backend.
/Users/hoangvo/opt/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hoangvo/opt/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hoangvo/opt/anaconda3/envs/deep-learning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
#use socket server to create communiation
sio = socketio.Server()

app = Flask(__name__) #'__main__'
speed_limit = 10
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img



In [3]:
@sio.on('telemetry')
def telemetry(sid, data):
    speed = float(data['speed'])
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    image = img_preprocess(image)
    image = np.array([image])
    steering_angle = float(model.predict(image))
    throttle = 1.0 - speed/speed_limit
    print('{} {} {}'.format(steering_angle, throttle, speed))
    send_control(steering_angle, throttle)




In [4]:
#from package eventlet
@sio.on('connect') #message, connect
def connect(sid, environ):
    print('Connected')
    send_control(0, 0)

def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__()
    })

In [ ]:

model = load_model('model.h5')
app = socketio.Middleware(sio, app) #from socketio package
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)




Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


(8799) wsgi starting up on http://0.0.0.0:4567
(8799) accepted ('127.0.0.1', 62624)
(8799) accepted ('127.0.0.1', 62625)
127.0.0.1 - - [11/Jul/2020 12:07:06] "GET / HTTP/1.1" 404 380 0.000458
(8799) accepted ('127.0.0.1', 62626)
127.0.0.1 - - [11/Jul/2020 12:07:06] "GET /favicon.ico HTTP/1.1" 404 380 0.000759
127.0.0.1 - - [11/Jul/2020 12:07:21] "GET / HTTP/1.1" 404 380 0.000436
127.0.0.1 - - [11/Jul/2020 12:07:22] "GET / HTTP/1.1" 404 380 0.000356


In [ ]:
!python Drive.py